## 0. Import Library

In [ ]:
! pip install category_encoders

In [71]:
# data analysis related library
import pandas as pd
import numpy as np

# some visualization related library
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
from matplotlib.colors import LogNorm
import seaborn as sns

# category_encoders and warning related library
from warnings import simplefilter
simplefilter(action='ignore')
import category_encoders

# sklearn related ML library
from sklearn import metrics
from sklearn.metrics import roc_curve, RocCurveDisplay
from sklearn.metrics import mean_absolute_error
from sklearn.utils import resample
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn import model_selection
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import auc
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# other useful libray
import missingno as msno
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from pylab import subplots_adjust
import copy
import torch
import torch.nn as nn
import torch.optim as optim

In [72]:
# data analysis related library
import pandas as pd
import numpy as np

# some visualization related library
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
from matplotlib.colors import LogNorm
import seaborn as sns

# category_encoders and warning related library
from warnings import simplefilter
simplefilter(action='ignore')
import category_encoders

# sklearn related ML library
from sklearn import metrics
from sklearn.metrics import roc_curve, RocCurveDisplay
from sklearn.metrics import mean_absolute_error
from sklearn.utils import resample
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn import model_selection
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import auc
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# other useful libray
import missingno as msno
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from pylab import subplots_adjust
import copy
import torch
import torch.nn as nn
import torch.optim as optim

## 1. Read Data

In [73]:
health_data = pd.read_csv("framingham.csv")

In [ ]:
health_data.info()

In [ ]:
health_data.shape

In [ ]:
health_data.head()

In [ ]:
health_data.describe()

In [ ]:
health_data.isna().sum().sum()

In [ ]:
health_data.isnull().sum().sort_values(ascending=False)

In [ ]:
(health_data.isnull().sum() / health_data.isnull().count()).sort_values(ascending=False)

In [ ]:
msno.dendrogram(health_data)
plt.show()

In [ ]:
msno.heatmap(health_data)
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
msno.matrix(health_data, color=(0.6, 0.2, 0.8))  # 使用不同的RGB配色
plt.show()

In [ ]:
msno.matrix(health_data,color=(47/255,127/255,255/255))
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# 计算每个列的缺失值数量
missing_counts = health_data.isnull().sum()

# 将数据拆分成两部分
num_columns = len(missing_counts)
half = num_columns // 2
cols_part1 = missing_counts[:half]
cols_part2 = missing_counts[half:]

# 创建横向柱状图
fig, axes = plt.subplots(1, 2, figsize=(14, 6))  # 1行2列子图

# 绘制第一个子图（第一部分的缺失值）
axes[0].barh(cols_part1.index, cols_part1.values, color=(255/255, 151/255, 0/255))
#axes[0].set_title("Missing Values")
axes[0].set_xlabel("Count")

# 绘制第二个子图（第二部分的缺失值）
axes[1].barh(cols_part2.index, cols_part2.values, color=(255/255, 151/255, 0/255))
#axes[1].set_title("Missing Values")
axes[1].set_xlabel("Count")

# 调整布局
plt.tight_layout()
plt.show()


In [ ]:
msno.bar(health_data.sample(1000), color=(255/255,151/225,0/255))
plt.show()

## 2. Data Cleaning, Analysis and Featrue Selection

### 2.1 Fill Missing Value

In [ ]:
health_data.TenYearCHD.value_counts()

In [ ]:
# search the numerical features
numerical_features = health_data.columns[health_data.dtypes != object]
list(numerical_features)

In [ ]:
num_features = len(health_data.columns)  
num_rows = 2  
num_cols = (num_features + num_rows - 1) // num_rows  

fig, axes = plt.subplots(num_rows, num_cols, figsize=(num_cols * 4, num_rows * 4))


axes = axes.flatten()

for i, col in enumerate(health_data.columns):
    sns.histplot(x=col, data=health_data, ax=axes[i], color=(159/255, 57/255, 222/255))

    
    axes[i].set_xlabel(col, fontsize=16)  
    axes[i].set_ylabel("Count", fontsize=16)  


plt.tight_layout()
plt.show()




In [90]:
health_data_filled_with_mean = pd.DataFrame()
for col in health_data.columns:
    if health_data[col].dtype == 'float64':
        health_data_filled_with_mean[col] = health_data[col].fillna(health_data[col].mean())
    elif set(health_data[col].dropna().unique()).issubset({0, 1}):
        health_data_filled_with_mean[col] = health_data[col].fillna(health_data[col].median())
    else:
        health_data_filled_with_mean[col] = health_data[col]



In [ ]:
num_features = len(health_data_filled_with_mean.columns)  
num_rows = 2  
num_cols = (num_features + num_rows - 1) // num_rows 

fig, axes = plt.subplots(num_rows, num_cols, figsize=(num_cols * 4, num_rows * 4))

axes = axes.flatten()

for i, col in enumerate(health_data_filled_with_mean.columns):
    sns.histplot(x=col, data=health_data_filled_with_mean, ax=axes[i], color=(159/255, 57/255, 222/255))

    axes[i].set_xlabel(col, fontsize=16)
    axes[i].set_ylabel("Count", fontsize=16)

    axes[i].tick_params(axis='x', labelsize=14)
    axes[i].tick_params(axis='y', labelsize=14)

plt.tight_layout()
plt.show()

In [ ]:
msno.matrix(health_data_filled_with_mean,color=(47/255,127/255,255/255))
msno.matrix(health_data_filled_with_mean, color=(0.6, 0.2, 0.8))  
plt.show()

In [ ]:
health_data_filled_with_mean.isnull().sum().sort_values()

In [ ]:
health_data_filled_with_mean.info()

In [95]:
health_data_filled_with_mean['BPMeds'] = health_data_filled_with_mean['BPMeds'].astype('int64')

In [ ]:
health_data_filled_with_mean.info()

### 2.2 Handle outliers

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams.update({
    'font.size': 18,            
    'font.weight': 'normal',    
    'axes.labelweight': 'normal',
    'axes.titleweight': 'normal'
})


fig = plt.figure(figsize=(35, 40))
fig.set_facecolor('white')  

index = 1
target_col = 'TenYearCHD'


box_color = (0/255, 100/255, 200/255)  


for k in health_data_filled_with_mean:
    if health_data_filled_with_mean[k].dtype == 'float64':
        ax = plt.subplot(len(numerical_features), 2, index)

        
        sns.boxplot(
            x=health_data_filled_with_mean[target_col].astype(str),  
            y=health_data_filled_with_mean[k],
            color=box_color  
        )

    
        plt.title(f'Box Plot of {k} by {target_col}', fontsize=12, fontweight='normal')
        plt.xlabel('TenYearCHD', fontsize=12, fontweight='normal')
        plt.ylabel(k, fontsize=12, fontweight='normal')

        
        ax.yaxis.grid(True, linestyle='--', alpha=0.7)

      
        for spine in ax.spines.values():
            spine.set_edgecolor('darkblue')  
            spine.set_linewidth(2)  
            spine.set_linestyle('-')  

        
        ax.spines['top'].set_visible(False)  
        ax.spines['right'].set_visible(False)  
        ax.spines['left'].set_color('black')  
        ax.spines['bottom'].set_color('black')  

        index += 1

plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

fig = plt.figure(figsize=(35, 40))
fig.set_facecolor((235/255, 235/255, 235/255))

index = 1
target_col = 'TenYearCHD'

for k in health_data_filled_with_mean:
    if health_data_filled_with_mean[k].dtype == 'float64':
        plt.subplot(len(numerical_features), 2, index)
        sns.boxplot(
            x=health_data_filled_with_mean[target_col].astype(str),  # 将目标列转换为字符串
            y=health_data_filled_with_mean[k],
            palette={'0': (0/255, 219/255, 197/255), '1': (255/255, 99/255, 132/255)}
            
        )
        plt.title(f'Box Plot of {k} by {target_col}')
        plt.xlabel('TenYearCHD')
        plt.ylabel(k)
        index += 1

plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

fig = plt.figure(figsize=(35, 40))
fig.set_facecolor((235/255, 235/255, 235/255))

index = 1
target_col = 'TenYearCHD'

for k in health_data_filled_with_mean:
    if health_data_filled_with_mean[k].dtype == 'float64':
        plt.subplot(len(numerical_features), 2, index)
        sns.boxplot(
            x=health_data_filled_with_mean[target_col].astype(str),  
            y=health_data_filled_with_mean[k],
            palette={'0': (0/255, 219/255, 197/255), '1': (255/255, 99/255, 132/255)}
        )
        plt.title(f'Box Plot of {k} by {target_col}', fontsize=20)  
        plt.xlabel('TenYearCHD', fontsize=18)  
        plt.ylabel(k, fontsize=18)  
        plt.xticks(fontsize=16)  
        plt.yticks(fontsize=16)  
        index += 1

plt.tight_layout()
plt.show()


### 2.2.1 Handle outliers for TenYearCHD = 0

In [100]:
group_0 = health_data_filled_with_mean[health_data_filled_with_mean['TenYearCHD'] == 0]

In [ ]:
fig = plt.figure(figsize=(35,20))
fig.set_facecolor((235/255,235/255,235/255))
index = 1
for k in group_0:
    if group_0[k].dtype=='float64':
        plt.subplot(4, 6, index)
        sns.boxplot(group_0[k], color=(0/255,219/255,197/255))
        index += 1
plt.tight_layout()
plt.show()

In [102]:
for col in group_0.columns:
    if col != 'TenYearCHD' and group_0[col].dtype == 'float64':
        Q1 = group_0[col].quantile(0.25)
        Q3 = group_0[col].quantile(0.75)
        IQR = Q3 - Q1

        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        group_0[col] = np.where(group_0[col] < lower_bound, lower_bound, group_0[col])
        group_0[col] = np.where(group_0[col] > upper_bound, upper_bound, group_0[col])

In [ ]:
fig = plt.figure(figsize=(35,20))
fig.set_facecolor((235/255,235/255,235/255))
index = 1
for k in group_0:
    if group_0[k].dtype=='float64':
        plt.subplot(4, 6, index)
        sns.boxplot(group_0[k], color=(0/255,219/255,197/255))
        index += 1
plt.tight_layout()
plt.show()

### 2.2.2 For TenYearCHD = 1

In [104]:
group_1 = health_data_filled_with_mean[health_data_filled_with_mean['TenYearCHD'] == 1]

In [ ]:
fig = plt.figure(figsize=(35,20))
fig.set_facecolor((235/255,235/255,235/255))
index = 1
for k in group_1:
    if group_1[k].dtype=='float64':
        plt.subplot(4, 6, index)
        sns.boxplot(group_1[k], color=(0/255,219/255,197/255))
        index += 1
plt.tight_layout()
plt.show()

In [106]:
for col in group_1.columns:
    if col != 'TenYearCHD' and group_1[col].dtype == 'float64':
        Q1 = group_1[col].quantile(0.25)
        Q3 = group_1[col].quantile(0.75)
        IQR = Q3 - Q1

        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        group_1[col] = np.where(group_1[col] < lower_bound, lower_bound, group_1[col])
        group_1[col] = np.where(group_1[col] > upper_bound, upper_bound, group_1[col])

In [ ]:
fig = plt.figure(figsize=(35,20))
fig.set_facecolor((235/255,235/255,235/255))
index = 1
for k in group_1:
    if group_1[k].dtype=='float64':
        plt.subplot(4, 6, index)
        sns.boxplot(group_1[k], color=(0/255,219/255,197/255))
        index += 1
plt.tight_layout()
plt.show()

As the number of 1 is small, we keep all the outliers to let the model learn all the features

In [108]:
health_data_filled_with_mean = pd.concat([group_0, group_1])

### 2.2.3 Final box plot

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams.update({
    'font.size': 18,            
    'font.weight': 'normal',    
    'axes.labelweight': 'normal',
    'axes.titleweight': 'normal'
})

fig = plt.figure(figsize=(35, 40))
fig.set_facecolor('white')  

index = 1
target_col = 'TenYearCHD'

box_color = (0/255, 100/255, 200/255)  

for k in health_data_filled_with_mean:
    if health_data_filled_with_mean[k].dtype == 'float64':
        ax = plt.subplot(len(numerical_features), 2, index)

  
        sns.boxplot(
            x=health_data_filled_with_mean[target_col].astype(str),  
            y=health_data_filled_with_mean[k],
            color=box_color  
        )

        plt.title(f'Box Plot of {k} by {target_col}', fontsize=12, fontweight='normal')
        plt.xlabel('TenYearCHD', fontsize=12, fontweight='normal')
        plt.ylabel(k, fontsize=12, fontweight='normal')

        ax.yaxis.grid(True, linestyle='--', alpha=0.7)


        for spine in ax.spines.values():
            spine.set_edgecolor('darkblue')  
            spine.set_linewidth(2)  
            spine.set_linestyle('-')  

        ax.spines['top'].set_visible(False)  
        ax.spines['right'].set_visible(False)  
        ax.spines['left'].set_color('black')  
        ax.spines['bottom'].set_color('black')  

        index += 1

plt.tight_layout()
plt.show()


## 2.4 Visualization and Drop highly-correlated features

In [ ]:
health_data_filled_with_mean.info()

In [ ]:
corr_matrix = health_data_filled_with_mean.corr()

mask = np.triu(np.ones_like(corr_matrix, dtype=bool))

plt.figure(figsize=(12, 11))
sns.heatmap(corr_matrix, mask=mask, cmap="YlGnBu", annot=True, fmt='.2f')
plt.title("Correlation Heatmap")
plt.show()

In [ ]:
plt.figure(figsize=(15,15))
features_correlation = abs(corr_matrix['TenYearCHD']).sort_values(ascending=False)

size = [
    features_correlation.iloc[1], features_correlation.iloc[2],
    features_correlation.iloc[3], features_correlation.iloc[4],
    features_correlation.iloc[5], features_correlation.iloc[6],
    features_correlation.iloc[7], features_correlation.iloc[8],
    features_correlation.iloc[9], features_correlation.iloc[10],
    features_correlation.iloc[11], features_correlation.iloc[12],
    features_correlation.iloc[13], features_correlation.iloc[14],
    features_correlation.iloc[15],
]

labels = [
    features_correlation.index[1], features_correlation.index[2],
    features_correlation.index[3], features_correlation.index[4],
    features_correlation.index[5], features_correlation.index[6],
    features_correlation.index[7], features_correlation.index[8],
    features_correlation.index[9], features_correlation.index[10],
    features_correlation.index[11], features_correlation.index[12],
    features_correlation.index[13], features_correlation.index[14],
    features_correlation.index[15],
]

explode = [0.1] * 5 + [0] * (len(size) - 5)

plt.figure(figsize=(8, 8))
plt.pie(size, labels=labels, explode=explode, autopct='%1.2f%%')
plt.title('Correlation with TenYearCHD (Top 5 Highlighted)', fontsize=16, fontweight='bold')
plt.show()



CurrentSmoker has a strong correlation with cigsPerDay (approximately 0.77), indicating a significant linear relationship between the two features. Since cigsPerDay is a float feature that captures more granular information about smoking intensity (e.g., the number of cigarettes smoked per day), we propose retaining cigsPerDay as the primary smoking-related feature and removing CurrentSmoker to reduce feature redundancy.

Also, we think that diaBP and sysBP can presents better than int64 feature prevalentHyp.

In [113]:
health_data_filled_with_mean_cleaned = health_data_filled_with_mean.drop('currentSmoker', axis=1)
health_data_filled_with_mean_cleaned = health_data_filled_with_mean_cleaned.drop('prevalentHyp', axis=1)

In [ ]:
corr_matrix = health_data_filled_with_mean_cleaned.corr()

mask = np.triu(np.ones_like(corr_matrix, dtype=bool))

plt.figure(figsize=(12, 11))
sns.heatmap(corr_matrix, mask=mask, cmap="YlGnBu", annot=True, fmt='.2f')
plt.title("Correlation Heatmap")
plt.show()

In [ ]:
health_data_filled_with_mean_cleaned.info()

### 2.5 Excute Balance and Scale

In [ ]:
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(health_data_filled_with_mean_cleaned)
scaled_data = pd.DataFrame(scaled_data, columns=health_data_filled_with_mean_cleaned.columns)
scaled_data.info()

from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

X = scaled_data.drop(columns=['TenYearCHD'])
y = scaled_data['TenYearCHD']

x_train, x_validation, y_train, y_validation = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

smote = SMOTE(random_state=42)
x_train_upsampled, y_train_upsampled = smote.fit_resample(x_train, y_train)

from collections import Counter
print('Original training dataset shape:', Counter(y_train))
print('Upsampled training dataset shape:', Counter(y_train_upsampled))

## 3. Training, Hyperparameters Tried and Evaluation

### 3.1 DT

In [ ]:
parameter_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2],
    'max_features': ['sqrt', 'log2']
}


# Perform grid search for hyperparameter optimization
grid_search = GridSearchCV(
    DecisionTreeClassifier(random_state=42),
    param_grid=parameter_grid,
    cv=5,
    n_jobs=-1,
)
grid_search.fit(x_train_upsampled, y_train_upsampled)
best_params = grid_search.best_params_

print(f"Optimal parameters: {best_params}")

# Train the best decision tree classifier
best_classifier = DecisionTreeClassifier(**best_params, random_state=42)
best_classifier.fit(x_train_upsampled, y_train_upsampled)

# Evaluate the model on the validation set
y_pred = best_classifier.predict(x_validation)
y_pred_proba = best_classifier.predict_proba(x_validation)[:, 1]

# Calculate metrics
acc = metrics.accuracy_score(y_validation, y_pred)
precision = metrics.precision_score(y_validation, y_pred)
recall = metrics.recall_score(y_validation, y_pred)
f1 = metrics.f1_score(y_validation, y_pred)
auc = metrics.roc_auc_score(y_validation, y_pred_proba)
fpr, tpr, _ = metrics.roc_curve(y_validation, y_pred_proba)

# Store results in a DataFrame
results = [{
    'Label': '8:1:1',
    'Accuracy': "{:.3f}".format(acc),
    'Precision': "{:.3f}".format(precision),
    'Recall': "{:.3f}".format(recall),
    'ROC AUC': "{:.3f}".format(auc),
    'F1 Score': "{:.3f}".format(f1)
}]

df = pd.DataFrame(results)
print(df[['Label', 'Accuracy', 'Precision', 'Recall', 'ROC AUC', 'F1 Score']])

# Plot the ROC Curve
fig, ax = plt.subplots(1, 1, figsize=(10, 6))
ax.plot(fpr, tpr, label=f"ROC AUC = {auc:.3f}", color='b')
ax.plot([0, 1], [0, 1], 'k--', lw=2, alpha=0.7)
ax.legend(loc="lower right")
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')
ax.set_title("ROC Curve for Optimized Decision Tree")
plt.show()

In [ ]:
import shap
import matplotlib.pyplot as plt

explainer = shap.TreeExplainer(best_classifier)

shap_values = explainer.shap_values(x_validation)

shap_values_target = shap_values[:, :, 1]

print(f"Shape of SHAP values for target class: {shap_values_target.shape}")
print(f"Shape of x_validation: {x_validation.shape}")

shap.summary_plot(shap_values_target, x_validation, plot_type="bar")

shap.summary_plot(shap_values_target, x_validation)

sample_index = 0
shap.force_plot(
    explainer.expected_value[1],
    shap_values_target[sample_index],
    x_validation.iloc[sample_index],
    matplotlib=True
)


### 3.2 LR

In [ ]:
parameter_grid = {
    'C': [0.01, 0.1, 1, 10, 100],  
    'solver': ['lbfgs', 'liblinear'], 
    'max_iter': [1000]  
}

grid_search = GridSearchCV(
    LogisticRegression(random_state=42),
    param_grid=parameter_grid,
    cv=5,
    n_jobs=-1,
)
grid_search.fit(x_train_upsampled, y_train_upsampled)
best_params = grid_search.best_params_

print(f"Optimal parameters: {best_params}")

best_classifier = LogisticRegression(**best_params, random_state=42)
best_classifier.fit(x_train_upsampled, y_train_upsampled)

y_pred = best_classifier.predict(x_validation)
y_pred_proba = best_classifier.predict_proba(x_validation)[:, 1]

acc = metrics.accuracy_score(y_validation, y_pred)
precision = metrics.precision_score(y_validation, y_pred)
recall = metrics.recall_score(y_validation, y_pred)
f1 = metrics.f1_score(y_validation, y_pred)
auc = metrics.roc_auc_score(y_validation, y_pred_proba)
fpr, tpr, _ = metrics.roc_curve(y_validation, y_pred_proba)

results = [{
    'Label': '8:1:1',
    'Accuracy': "{:.3f}".format(acc),
    'Precision': "{:.3f}".format(precision),
    'Recall': "{:.3f}".format(recall),
    'ROC AUC': "{:.3f}".format(auc),
    'F1 Score': "{:.3f}".format(f1)
}]

df = pd.DataFrame(results)
print(df[['Label', 'Accuracy', 'Precision', 'Recall', 'ROC AUC', 'F1 Score']])

fig, ax = plt.subplots(1, 1, figsize=(10, 6))
ax.plot(fpr, tpr, label=f"ROC AUC = {auc:.3f}", color='b')
ax.plot([0, 1], [0, 1], 'k--', lw=2, alpha=0.7)
ax.legend(loc="lower right")
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')
ax.set_title("ROC Curve for Optimized Logistic Regression")
plt.show()

### 3.3 NB

In [ ]:
parameter_grid = {
    'alpha': [0.1, 0.5, 1, 2, 5, 10],
    'binarize': [0, 0.5, 1, 2]
}

grid_search = GridSearchCV(
    BernoulliNB(),
    param_grid=parameter_grid,
    cv=5,
    n_jobs=-1,
    scoring='accuracy'
)

grid_search.fit(x_train_upsampled, y_train_upsampled)
best_params = grid_search.best_params_

print(f"Optimal parameters: {best_params}")

best_nb = BernoulliNB(**best_params)
best_nb.fit(x_train_upsampled, y_train_upsampled)

y_pred = best_nb.predict(x_validation)
y_pred_proba = best_nb.predict_proba(x_validation)[:, 1]

acc = metrics.accuracy_score(y_validation, y_pred)
precision = metrics.precision_score(y_validation, y_pred)
recall = metrics.recall_score(y_validation, y_pred)
f1 = metrics.f1_score(y_validation, y_pred)
auc = metrics.roc_auc_score(y_validation, y_pred_proba)
fpr, tpr, _ = metrics.roc_curve(y_validation, y_pred_proba)

results = [{
    'Label': '8:1:1',
    'Accuracy': "{:.3f}".format(acc),
    'Precision': "{:.3f}".format(precision),
    'Recall': "{:.3f}".format(recall),
    'ROC AUC': "{:.3f}".format(auc),
    'F1 Score': "{:.3f}".format(f1)
}]

df = pd.DataFrame(results)
print(df[['Label', 'Accuracy', 'Precision', 'Recall', 'ROC AUC', 'F1 Score']])

# Plot the ROC Curve
fig, ax = plt.subplots(1, 1, figsize=(10, 6))
ax.plot(fpr, tpr, label=f"ROC AUC = {auc:.3f}", color='b')
ax.plot([0, 1], [0, 1], 'k--', lw=2, alpha=0.7)
ax.legend(loc="lower right")
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')
ax.set_title("ROC Curve for Optimized BernoulliNB Model")
plt.show()

### 3.4 KNN

In [ ]:
parameter_grid = {
    'n_neighbors': [80, 100, 200],
    'p': [1, 2],  # 1: Manhattan distance, 2: Euclidean distance
    'weights': ['uniform', 'distance']
}

grid_search = GridSearchCV(
    KNeighborsClassifier(),
    param_grid=parameter_grid,
    cv=5,
    n_jobs=-1,
    scoring='accuracy'
)
grid_search.fit(x_train_upsampled, y_train_upsampled)
best_params = grid_search.best_params_

print(f"Optimal parameters: {best_params}")

# Train the optimal classifier
knn_optimal = KNeighborsClassifier(**best_params)
knn_optimal.fit(x_train_upsampled, y_train_upsampled)

# Predict on validation set
y_pred = knn_optimal.predict(x_validation)
y_pred_proba = knn_optimal.predict_proba(x_validation)[:, 1]

# Calculate metrics
acc = metrics.accuracy_score(y_validation, y_pred)
precision = metrics.precision_score(y_validation, y_pred)
recall = metrics.recall_score(y_validation, y_pred)
f1 = metrics.f1_score(y_validation, y_pred)
auc = metrics.roc_auc_score(y_validation, y_pred_proba)
fpr, tpr, _ = metrics.roc_curve(y_validation, y_pred_proba)

results = [{
    'Label': '8:1:1',
    'Accuracy': "{:.3f}".format(acc),
    'Precision': "{:.3f}".format(precision),
    'Recall': "{:.3f}".format(recall),
    'ROC AUC': "{:.3f}".format(auc),
    'F1 Score': "{:.3f}".format(f1)
}]


# Create DataFrame for metrics
df_metrics = pd.DataFrame(results)
print(df_metrics[['Label', 'Accuracy', 'Precision', 'Recall', 'ROC AUC', 'F1 Score']])

# Plot the ROC Curve
fig, ax = plt.subplots(1, 1, figsize=(10, 6))
ax.plot(fpr, tpr, label=f"ROC AUC = {auc:.3f}", color='b')
ax.plot([0, 1], [0, 1], 'k--', lw=2, alpha=0.7)
ax.legend(loc="lower right")
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')
ax.set_title("ROC Curve for Optimized KNN Model")
plt.show()

### 3.5 GB

In [ ]:
# parameter_grid = {
#     'n_estimators': [10, 100, 1000, 1200],
#     'learning_rate': [0.1, 0.2, 0.4, 0.8]
# }

parameter_grid = {
    'n_estimators': [10, 100, 500],
    'learning_rate': [0.1, 0.2, 0.4]
}

grid_search = GridSearchCV(
    GradientBoostingClassifier(random_state=42),
    param_grid=parameter_grid,
    cv=5,
    n_jobs=-1,
    scoring='accuracy',
)

grid_search.fit(x_train_upsampled, y_train_upsampled)
best_params = grid_search.best_params_

print(f"Optimal parameters: {best_params}")

gb_optimal = GradientBoostingClassifier(**best_params, random_state=42)
gb_optimal.fit(x_train_upsampled, y_train_upsampled)

y_pred = gb_optimal.predict(x_validation)
y_pred_proba = gb_optimal.predict_proba(x_validation)[:, 1]

acc = metrics.accuracy_score(y_validation, y_pred)
precision = metrics.precision_score(y_validation, y_pred)
recall = metrics.recall_score(y_validation, y_pred)
f1 = metrics.f1_score(y_validation, y_pred)
auc = metrics.roc_auc_score(y_validation, y_pred_proba)
fpr, tpr, _ = metrics.roc_curve(y_validation, y_pred_proba)

results = [{
    'Label': '8:1:1',
    'Accuracy': "{:.3f}".format(acc),
    'Precision': "{:.3f}".format(precision),
    'Recall': "{:.3f}".format(recall),
    'ROC AUC': "{:.3f}".format(auc),
    'F1 Score': "{:.3f}".format(f1)
}]

df_metrics = pd.DataFrame(results)
print(df_metrics[['Label', 'Accuracy', 'Precision', 'Recall', 'ROC AUC', 'F1 Score']])

# Plot the ROC Curve
fig, ax = plt.subplots(1, 1, figsize=(10, 6))
ax.plot(fpr, tpr, label=f"ROC AUC = {auc:.3f}", color='b')
ax.plot([0, 1], [0, 1], 'k--', lw=2, alpha=0.7)
ax.legend(loc="lower right")
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')
ax.set_title("ROC Curve for Optimized Gradient Boosting Model")
plt.show()

### 3.6 ElasticNet

In [ ]:
parameter_grid = {
    'penalty': ['elasticnet'],
    'solver': ['saga'],
    'C': [10, 50, 100,300, 500],
    'l1_ratio': [0.0001, 0.001, 0.01],
    'max_iter': [1000],
    'tol': [1e-4, 1e-3, 1e-2],
    'class_weight': [None, 'balanced'],
}


grid_search = GridSearchCV(
    LogisticRegression(random_state=42),
    param_grid=parameter_grid,
    cv=5,
    n_jobs=-1,
    scoring='accuracy'
)
grid_search.fit(x_train_upsampled, y_train_upsampled)
best_params = grid_search.best_params_

print(f"Optimal parameters: {best_params}")

best_classifier = LogisticRegression(**best_params, random_state=42)
best_classifier.fit(x_train_upsampled, y_train_upsampled)

y_pred = best_classifier.predict(x_validation)
y_pred_proba = best_classifier.predict_proba(x_validation)[:, 1]

acc = metrics.accuracy_score(y_validation, y_pred)
precision = metrics.precision_score(y_validation, y_pred)
recall = metrics.recall_score(y_validation, y_pred)
f1 = metrics.f1_score(y_validation, y_pred)
auc = metrics.roc_auc_score(y_validation, y_pred_proba)
fpr, tpr, _ = metrics.roc_curve(y_validation, y_pred_proba)

results = [{
    'Label': '8:1:1',
    'Accuracy': "{:.3f}".format(acc),
    'Precision': "{:.3f}".format(precision),
    'Recall': "{:.3f}".format(recall),
    'ROC AUC': "{:.3f}".format(auc),
    'F1 Score': "{:.3f}".format(f1),
    'fpr': fpr,
    'tpr': tpr
}]

df = pd.DataFrame(results)
print(df[['Label', 'Accuracy', 'Precision', 'Recall', 'ROC AUC', 'F1 Score']])


# Plot the ROC Curve
fig, ax = plt.subplots(1, 1, figsize=(10, 6))
ax.plot(fpr, tpr, label=f"ROC AUC = {auc:.3f}", color='b')
ax.plot([0, 1], [0, 1], 'k--', lw=2, alpha=0.7)
ax.legend(loc="lower right")
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')
ax.set_title("ROC Curve for Optimized ElasticNet Logistic Regression")
plt.show()

### 3.7 MLP

In [ ]:
train_size = 0.8
test_size = 0.1
validation_size = 0.1

parameter_grid = {
    'hidden_layer_sizes': [
        (128,),
        (64,),
        (128, 64),
        (64, 32),
        (128, 64, 32)
    ],
    'activation': ['relu', 'tanh', 'logistic'],
    'learning_rate_init': [0.001, 0.01, 0.1],
    'alpha': [ 0.001, 0.01, 0.1]
}

grid_search = GridSearchCV(
    estimator=MLPClassifier(
        solver='adam',
        max_iter=200,
        random_state=42
    ),
    param_grid=parameter_grid,
    cv=5,
    n_jobs=-1,
    scoring='accuracy',
    verbose=2
)

grid_search.fit(x_train_upsampled, y_train_upsampled)


best_params = grid_search.best_params_
print(f"Optimal parameters: {best_params}")


mlp_optimal = MLPClassifier(
    hidden_layer_sizes=best_params['hidden_layer_sizes'],
    activation=best_params['activation'],
    solver='adam',
    learning_rate_init=best_params['learning_rate_init'],
    alpha=best_params['alpha'],
    max_iter=300,
    random_state=42
)
mlp_optimal.fit(x_train_upsampled, y_train_upsampled)


y_pred = mlp_optimal.predict(x_validation)
y_pred_proba = mlp_optimal.predict_proba(x_validation)[:, 1]


acc = metrics.accuracy_score(y_validation, y_pred)
precision = metrics.precision_score(y_validation, y_pred, zero_division=0)
recall = metrics.recall_score(y_validation, y_pred, zero_division=0)
f1 = metrics.f1_score(y_validation, y_pred, zero_division=0)
auc = metrics.roc_auc_score(y_validation, y_pred_proba)
fpr, tpr, _ = metrics.roc_curve(y_validation, y_pred_proba)


results = [{
    'Label': '8:1:1',
    'Accuracy': "{:.3f}".format(acc),
    'Precision': "{:.3f}".format(precision),
    'Recall': "{:.3f}".format(recall),
    'ROC AUC': "{:.3f}".format(auc),
    'F1 Score': "{:.3f}".format(f1),
    'fpr': fpr,
    'tpr': tpr
}]


df_metrics = pd.DataFrame(results)
print(df_metrics[['Label', 'Accuracy', 'Precision', 'Recall', 'ROC AUC', 'F1 Score']])


fig, ax = plt.subplots(figsize=(10,  6))
ax.plot(fpr, tpr, label=f"ROC AUC = {auc:.3f}", color='b')

ax.plot([0, 1], [0, 1], 'k--', lw=2, alpha=0.7)
ax.legend(loc="lower right")
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')
ax.set_title("ROC Curve for Optimized MLP Model")

plt.show()

### 3.8 Transformer

In [ ]:
# only nn.TransformerEncoder and nn.TransformerEncoderLayer
train_size = 0.8
test_size = 0.1
validation_size = 0.1


class SimpleTransformerClassifier(nn.Module):
    def __init__(self, n_features, embed_dim=32, nhead=4, num_layers=2, dim_feedforward=64, activation='relu'):
        super().__init__()
        self.n_features = n_features
        self.embed_dim = embed_dim

        self.feature_proj = nn.Linear(n_features, embed_dim)

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=embed_dim,
            nhead=nhead,
            dim_feedforward=dim_feedforward,
            batch_first=True,
            activation=activation
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        self.fc = nn.Linear(embed_dim, 1)
        self.sigmoid = nn.Sigmoid()
    def forward(self, x):
        batch_size = x.size(0)
        proj = self.feature_proj(x)
        proj = proj.view(batch_size, self.n_features, self.embed_dim)

        encoded = self.transformer_encoder(proj)

        pooled = encoded.mean(dim=1)
        logit = self.fc(pooled)
        prob = self.sigmoid(logit)
        return prob


class PyTorchTransformerClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self,
                 embed_dim=32, nhead=4, num_layers=2, dim_feedforward=64,
                 learning_rate=0.001, max_iter=200, device='cpu', activation='relu'):
        self.embed_dim = embed_dim
        self.nhead = nhead
        self.num_layers = num_layers
        self.dim_feedforward = dim_feedforward
        self.learning_rate = learning_rate
        self.max_iter = max_iter
        self.device = device
        self.activation = activation
        self.model_ = None
        self.n_features_ = None

    def fit(self, X, y):
        self.n_features_ = X.shape[1]
        X_tensor = torch.tensor(X, dtype=torch.float32).to(self.device)
        y_tensor = torch.tensor(y, dtype=torch.float32).view(-1,1).to(self.device)

        self.model_ = SimpleTransformerClassifier(
            n_features=self.n_features_,
            embed_dim=self.embed_dim,
            nhead=self.nhead,
            num_layers=self.num_layers,
            dim_feedforward=self.dim_feedforward,
            activation=self.activation
        ).to(self.device)

        self.model_.feature_proj = nn.Linear(self.n_features_, self.n_features_*self.embed_dim).to(self.device)

        optimizer = optim.Adam(self.model_.parameters(), lr=self.learning_rate)
        criterion = nn.BCELoss()

        self.model_.train()
        for epoch in range(self.max_iter):
            optimizer.zero_grad()
            outputs = self.model_(X_tensor)
            loss = criterion(outputs, y_tensor)
            loss.backward()
            optimizer.step()
        return self

    def predict_proba(self, X):
        self.model_.eval()
        with torch.no_grad():
            X_tensor = torch.tensor(X, dtype=torch.float32).to(self.device)
            pred = self.model_(X_tensor).cpu().numpy()
        return np.hstack([1 - pred, pred])

    def predict(self, X):
        proba = self.predict_proba(X)[:, 1]
        return (proba >= 0.5).astype(int)




transformer_optimal = PyTorchTransformerClassifier(
    embed_dim=16,
    nhead=4,
    num_layers=1,
    dim_feedforward=64,
    learning_rate=1e-3,
    max_iter=200,
    activation='relu',
    device='cpu'
)

x_train_upsampled = x_train_upsampled.values if hasattr(x_train_upsampled, 'values') else x_train_upsampled
x_validation = x_validation.values if hasattr(x_validation, 'values') else x_validation
y_train_upsampled = y_train_upsampled.values if hasattr(y_train_upsampled, 'values') else y_train_upsampled
y_validation = y_validation.values if hasattr(y_validation, 'values') else y_validation

transformer_optimal.fit(x_train_upsampled, y_train_upsampled)

y_pred = transformer_optimal.predict(x_validation)
y_pred_proba = transformer_optimal.predict_proba(x_validation)[:, 1]

acc = metrics.accuracy_score(y_validation, y_pred)
precision = metrics.precision_score(y_validation, y_pred, zero_division=0)
recall = metrics.recall_score(y_validation, y_pred, zero_division=0)
f1 = metrics.f1_score(y_validation, y_pred, zero_division=0)
auc = metrics.roc_auc_score(y_validation, y_pred_proba)
fpr, tpr, _ = metrics.roc_curve(y_validation, y_pred_proba)

results = [{
    'Label': "8:1:1",
    'Accuracy': "{:.3f}".format(acc),
    'Precision': "{:.3f}".format(precision),
    'Recall': "{:.3f}".format(recall),
    'ROC AUC': "{:.3f}".format(auc),
    'F1 Score': "{:.3f}".format(f1),
    'fpr': fpr,
    'tpr': tpr
}]

df_metrics = pd.DataFrame(results)
print(df_metrics[['Label', 'Accuracy', 'Precision', 'Recall', 'ROC AUC', 'F1 Score']])

fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(fpr, tpr, label=f"ROC AUC = {auc:.3f}", color='b')
ax.plot([0, 1], [0, 1], 'k--', lw=2, alpha=0.7)
ax.legend(loc="lower right")
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')
ax.set_title("ROC Curve for Optimized Transformer Model")
plt.show()


### 3.9 KAN

In [ ]:
train_size = 0.8
test_size = 0.1
validation_size = 0.1

class UnivariateMLP(nn.Module):
    def __init__(self, hidden_dim=32, activation='relu'):
        super().__init__()
        act_map = {
            'relu': nn.ReLU(),
            'tanh': nn.Tanh(),
            'gelu': nn.GELU()
        }
        self.net = nn.Sequential(
            nn.Linear(1, hidden_dim),
            act_map.get(activation, nn.ReLU()),
            nn.Linear(hidden_dim, hidden_dim),
            act_map.get(activation, nn.ReLU()),
            nn.Linear(hidden_dim, 1)
        )

    def forward(self, x):
        # x: [batch_size, 1]
        return self.net(x)

class KANModel(nn.Module):
    def __init__(self, n_features, Q=2, hidden_dim=32, activation='relu'):
        super().__init__()
        self.n_features = n_features
        self.Q = Q
        self.hidden_dim = hidden_dim

        self.psi_funcs = nn.ModuleList([
            nn.ModuleList([UnivariateMLP(hidden_dim, activation) for _ in range(n_features)])
            for _ in range(Q)
        ])

        # phi_q
        self.phi_funcs = nn.ModuleList([
            UnivariateMLP(hidden_dim, activation) for _ in range(Q)
        ])

        self.fc = nn.Linear(Q, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # x: [batch_size, n_features]
        batch_size = x.size(0)
        z_list = []
        for q in range(self.Q):
            sum_over_p = 0
            for p in range(self.n_features):
                xp = x[:, p:p+1]
                val = self.psi_funcs[q][p](xp)
                sum_over_p = sum_over_p + val
            phi_val = self.phi_funcs[q](sum_over_p) # [batch_size,1]
            z_list.append(phi_val)

        Z = torch.cat(z_list, dim=1)

        logit = self.fc(Z) # [batch_size,1]
        prob = self.sigmoid(logit)
        return prob


class PyTorchKANClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, Q=2, hidden_dim=32, activation='relu',
                 learning_rate=0.001, max_iter=100, device='cpu'):
        self.Q = Q
        self.hidden_dim = hidden_dim
        self.activation = activation
        self.learning_rate = learning_rate
        self.max_iter = max_iter
        self.device = device
        self.model_ = None
        self.n_features_ = None

    def fit(self, X, y):
        self.n_features_ = X.shape[1]
        X_tensor = torch.tensor(X, dtype=torch.float32).to(self.device)
        y_tensor = torch.tensor(y, dtype=torch.float32).view(-1,1).to(self.device)

        self.model_ = KANModel(
            n_features=self.n_features_,
            Q=self.Q,
            hidden_dim=self.hidden_dim,
            activation=self.activation
        ).to(self.device)

        optimizer = optim.Adam(self.model_.parameters(), lr=self.learning_rate)
        criterion = nn.BCELoss()

        self.model_.train()
        for epoch in range(self.max_iter):
            optimizer.zero_grad()
            outputs = self.model_(X_tensor)
            loss = criterion(outputs, y_tensor)
            loss.backward()
            optimizer.step()
        return self

    def predict_proba(self, X):
        self.model_.eval()
        with torch.no_grad():
            X_tensor = torch.tensor(X, dtype=torch.float32).to(self.device)
            pred = self.model_(X_tensor).cpu().numpy() # [batch_size,1]
        return np.hstack([1 - pred, pred])

    def predict(self, X):
        proba = self.predict_proba(X)[:, 1]
        return (proba >= 0.5).astype(int)


kan_optimal = PyTorchKANClassifier(
    Q=2,
    hidden_dim=64,
    activation='relu',
    learning_rate=1e-3,
    max_iter=1000,
    device='cpu'
)

x_train_upsampled = x_train_upsampled.values if hasattr(x_train_upsampled, 'values') else x_train_upsampled
x_validation = x_validation.values if hasattr(x_validation, 'values') else x_validation
y_train_upsampled = y_train_upsampled.values if hasattr(y_train_upsampled, 'values') else y_train_upsampled
y_validation = y_validation.values if hasattr(y_validation, 'values') else y_validation

kan_optimal.fit(x_train_upsampled, y_train_upsampled)


y_pred = kan_optimal.predict(x_validation)
y_pred_proba = kan_optimal.predict_proba(x_validation)[:, 1]

acc = metrics.accuracy_score(y_validation, y_pred)
precision = metrics.precision_score(y_validation, y_pred, zero_division=0)
recall = metrics.recall_score(y_validation, y_pred, zero_division=0)
f1 = metrics.f1_score(y_validation, y_pred, zero_division=0)
auc = metrics.roc_auc_score(y_validation, y_pred_proba)
fpr, tpr, _ = metrics.roc_curve(y_validation, y_pred_proba)

results = [{
    'Label': "8:1:1",
    'Accuracy': "{:.3f}".format(acc),
    'Precision': "{:.3f}".format(precision),
    'Recall': "{:.3f}".format(recall),
    'ROC AUC': "{:.3f}".format(auc),
    'F1 Score': "{:.3f}".format(f1),
    'fpr': fpr,
    'tpr': tpr
}]

df_metrics = pd.DataFrame(results)
print(df_metrics[['Label', 'Accuracy', 'Precision', 'Recall', 'ROC AUC', 'F1 Score']])

fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(fpr, tpr, label=f"ROC AUC = {auc:.3f}", color='b')
ax.plot([0, 1], [0, 1], 'k--', lw=2, alpha=0.7)
ax.legend(loc="lower right")
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')
ax.set_title("ROC Curve for Optimized KAN Model")
plt.show()

### 3.10 RNN

In [ ]:
train_size = 0.8
test_size = 0.1
validation_size = 0.1

class SimpleRNNClassifier(nn.Module):
    def __init__(self, n_features, hidden_size=32, num_layers=1, activation='relu'):
        super().__init__()
        self.n_features = n_features
        self.hidden_size = hidden_size
        self.num_layers = num_layers


        self.rnn = nn.RNN(input_size=1, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)


        self.fc = nn.Linear(hidden_size, 1)


        act_map = {
            'relu': nn.ReLU(),
            'tanh': nn.Tanh(),
            'gelu': nn.GELU()
        }
        self.activation = act_map.get(activation, nn.ReLU())

        self.sigmoid = nn.Sigmoid()

    def forward(self, x):

        batch_size = x.size(0)
        seq_len = self.n_features
        # reshape x => [batch_size, seq_len, 1]
        x_seq = x.view(batch_size, seq_len, 1)


        rnn_out, h = self.rnn(x_seq)  # rnn_out: [batch_size, seq_len, hidden_size]


        out = rnn_out[:, -1, :]

        logit = self.fc(out)
        prob = self.sigmoid(logit)
        return prob


class PyTorchRNNClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, hidden_size=32, num_layers=1, activation='relu',
                 learning_rate=0.001, max_iter=200, device='cpu'):
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.activation = activation
        self.learning_rate = learning_rate
        self.max_iter = max_iter
        self.device = device
        self.model_ = None
        self.n_features_ = None

    def fit(self, X, y):
        if hasattr(X, 'values'):
            X = X.values
        if hasattr(y, 'values'):
            y = y.values

        self.n_features_ = X.shape[1]
        X_tensor = torch.tensor(X, dtype=torch.float32).to(self.device)
        y_tensor = torch.tensor(y, dtype=torch.float32).view(-1,1).to(self.device)

        self.model_ = SimpleRNNClassifier(
            n_features=self.n_features_,
            hidden_size=self.hidden_size,
            num_layers=self.num_layers,
            activation=self.activation
        ).to(self.device)

        optimizer = optim.Adam(self.model_.parameters(), lr=self.learning_rate)
        criterion = nn.BCELoss()

        self.model_.train()
        for epoch in range(self.max_iter):
            optimizer.zero_grad()
            outputs = self.model_(X_tensor)
            loss = criterion(outputs, y_tensor)
            loss.backward()
            optimizer.step()
        return self

    def predict_proba(self, X):
        if hasattr(X, 'values'):
            X = X.values
        self.model_.eval()
        with torch.no_grad():
            X_tensor = torch.tensor(X, dtype=torch.float32).to(self.device)
            pred = self.model_(X_tensor).cpu().numpy()
        return np.hstack([1 - pred, pred])

    def predict(self, X):
        proba = self.predict_proba(X)[:, 1]
        return (proba >= 0.5).astype(int)


rnn_optimal = PyTorchRNNClassifier(
    hidden_size=64,
    num_layers=2,
    activation='relu',
    learning_rate=1e-3,
    max_iter=200,
    device='cpu'
)

x_train_upsampled = x_train_upsampled.values if hasattr(x_train_upsampled, 'values') else x_train_upsampled
x_validation = x_validation.values if hasattr(x_validation, 'values') else x_validation
y_train_upsampled = y_train_upsampled.values if hasattr(y_train_upsampled, 'values') else y_train_upsampled
y_validation = y_validation.values if hasattr(y_validation, 'values') else y_validation

rnn_optimal.fit(x_train_upsampled, y_train_upsampled)

y_pred = rnn_optimal.predict(x_validation)
y_pred_proba = rnn_optimal.predict_proba(x_validation)[:, 1]

acc = metrics.accuracy_score(y_validation, y_pred)
precision = metrics.precision_score(y_validation, y_pred, zero_division=0)
recall = metrics.recall_score(y_validation, y_pred, zero_division=0)
f1 = metrics.f1_score(y_validation, y_pred, zero_division=0)
auc = metrics.roc_auc_score(y_validation, y_pred_proba)
fpr, tpr, _ = metrics.roc_curve(y_validation, y_pred_proba)

results = [{
    'Label': "8:1:1",
    'Accuracy': "{:.3f}".format(acc),
    'Precision': "{:.3f}".format(precision),
    'Recall': "{:.3f}".format(recall),
    'ROC AUC': "{:.3f}".format(auc),
    'F1 Score': "{:.3f}".format(f1),
    'fpr': fpr,
    'tpr': tpr
}]

df_metrics = pd.DataFrame(results)
print(df_metrics[['Label', 'Accuracy', 'Precision', 'Recall', 'ROC AUC', 'F1 Score']])

fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(fpr, tpr, label=f"ROC AUC = {auc:.3f}", color='b')
ax.plot([0, 1], [0, 1], 'k--', lw=2, alpha=0.7)
ax.legend(loc="lower right")
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')
ax.set_title("ROC Curve for RNN Model")
plt.show()

### 3.11 CNN

In [ ]:
train_size = 0.8
test_size = 0.1
validation_size = 0.1

class SimpleCNNClassifier(nn.Module):
    def __init__(self, n_features, num_filters=32, kernel_size=3, activation='relu'):
        super().__init__()
        self.n_features = n_features
        self.num_filters = num_filters
        self.kernel_size = kernel_size

        self.conv1 = nn.Conv1d(in_channels=1, out_channels=num_filters, kernel_size=kernel_size, padding='same')


        act_map = {
            'relu': nn.ReLU(),
            'tanh': nn.Tanh(),
            'gelu': nn.GELU()
        }
        self.activation = act_map.get(activation, nn.ReLU())


        self.pool = nn.AdaptiveAvgPool1d(output_size=1)

        self.fc = nn.Linear(num_filters, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # x: [batch_size, n_features]
        batch_size = x.size(0)
        x = x.view(batch_size, 1, self.n_features)  # [batch_size, 1, n_features]

        out = self.conv1(x)  # [batch_size, num_filters, n_features]
        out = self.activation(out)
        out = self.pool(out) # [batch_size, num_filters, 1]

        out = out.squeeze(-1) # [batch_size, num_filters]
        logit = self.fc(out)  # [batch_size, 1]
        prob = self.sigmoid(logit)
        return prob


class PyTorchCNNClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, num_filters=32, kernel_size=3, activation='relu',
                 learning_rate=0.001, max_iter=100, batch_size=64, device='cpu'):
        self.num_filters = num_filters
        self.kernel_size = kernel_size
        self.activation = activation
        self.learning_rate = learning_rate
        self.max_iter = max_iter
        self.batch_size = batch_size
        self.device = device
        self.model_ = None
        self.n_features_ = None

    def fit(self, X, y):
        if hasattr(X, 'values'):
            X = X.values
        if hasattr(y, 'values'):
            y = y.values

        self.n_features_ = X.shape[1]
        X_tensor = torch.tensor(X, dtype=torch.float32).to(self.device)
        y_tensor = torch.tensor(y, dtype=torch.float32).view(-1,1).to(self.device)

        self.model_ = SimpleCNNClassifier(
            n_features=self.n_features_,
            num_filters=self.num_filters,
            kernel_size=self.kernel_size,
            activation=self.activation
        ).to(self.device)

        optimizer = optim.Adam(self.model_.parameters(), lr=self.learning_rate)
        criterion = nn.BCELoss()

        self.model_.train()
        for epoch in range(self.max_iter):
            for i in range(0, X_tensor.size(0), self.batch_size):
                batch_x = X_tensor[i:i+self.batch_size]
                batch_y = y_tensor[i:i+self.batch_size]

                optimizer.zero_grad()
                outputs = self.model_(batch_x)
                loss = criterion(outputs, batch_y)
                loss.backward()
                optimizer.step()

            if (epoch+1) % 10 == 0:
                print(f"Epoch [{epoch+1}/{self.max_iter}], Loss: {loss.item():.4f}")

        return self

    def predict_proba(self, X):
        if hasattr(X, 'values'):
            X = X.values
        self.model_.eval()
        with torch.no_grad():
            X_tensor = torch.tensor(X, dtype=torch.float32).to(self.device)
            pred = self.model_(X_tensor).cpu().numpy()
        return np.hstack([1 - pred, pred])

    def predict(self, X):
        proba = self.predict_proba(X)[:, 1]
        return (proba >= 0.5).astype(int)


cnn_clf = PyTorchCNNClassifier(
    num_filters=32,
    kernel_size=3,
    activation='relu',
    learning_rate=1e-3,
    max_iter=200,
    batch_size=64,
    device='cpu'
)

x_train_upsampled = x_train_upsampled.values if hasattr(x_train_upsampled, 'values') else x_train_upsampled
x_validation = x_validation.values if hasattr(x_validation, 'values') else x_validation
y_train_upsampled = y_train_upsampled.values if hasattr(y_train_upsampled, 'values') else y_train_upsampled
y_validation = y_validation.values if hasattr(y_validation, 'values') else y_validation

cnn_clf.fit(x_train_upsampled, y_train_upsampled)

y_pred = cnn_clf.predict(x_validation)
y_pred_proba = cnn_clf.predict_proba(x_validation)[:, 1]

acc = metrics.accuracy_score(y_validation, y_pred)
precision = metrics.precision_score(y_validation, y_pred, zero_division=0)
recall = metrics.recall_score(y_validation, y_pred, zero_division=0)
f1 = metrics.f1_score(y_validation, y_pred, zero_division=0)
auc = metrics.roc_auc_score(y_validation, y_pred_proba)
fpr, tpr, _ = metrics.roc_curve(y_validation, y_pred_proba)

results = [{
    'Label': "8:1:1",
    'Accuracy': "{:.3f}".format(acc),
    'Precision': "{:.3f}".format(precision),
    'Recall': "{:.3f}".format(recall),
    'ROC AUC': "{:.3f}".format(auc),
    'F1 Score': "{:.3f}".format(f1),
    'fpr': fpr,
    'tpr': tpr
}]

df_metrics = pd.DataFrame(results)
print(df_metrics[['Label', 'Accuracy', 'Precision', 'Recall', 'ROC AUC', 'F1 Score']])

fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(fpr, tpr, label=f"ROC AUC = {auc:.3f}", color='b')
ax.plot([0, 1], [0, 1], 'k--', lw=2, alpha=0.7)
ax.legend(loc="lower right")
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')
ax.set_title("ROC Curve for CNN Model")
plt.show()

### 3.12 ResNet

In [ ]:
train_size = 0.8
test_size = 0.1
validation_size = 0.1

class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, activation=nn.ReLU()):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size, padding='same', bias=False)
        self.bn1 = nn.BatchNorm1d(out_channels)
        self.activation = activation
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size, padding='same', bias=False)
        self.bn2 = nn.BatchNorm1d(out_channels)

        self.shortcut = nn.Sequential()
        if in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv1d(in_channels, out_channels, kernel_size=1, bias=False),
                nn.BatchNorm1d(out_channels)
            )

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.activation(out)

        out = self.conv2(out)
        out = self.bn2(out)

        shortcut = self.shortcut(x)

        out = out + shortcut
        out = self.activation(out)
        return out


class SimpleResNetClassifier(nn.Module):
    def __init__(self, n_features, num_blocks=2, hidden_dim=32, kernel_size=3, activation='relu'):
        super().__init__()
        act_map = {
            'relu': nn.ReLU(),
            'tanh': nn.Tanh(),
            'gelu': nn.GELU()
        }
        act = act_map.get(activation, nn.ReLU())

        self.conv_in = nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=hidden_dim, kernel_size=kernel_size, padding='same', bias=False),
            nn.BatchNorm1d(hidden_dim),
            act
        )

        layers = []
        in_channels = hidden_dim
        out_channels = hidden_dim
        for i in range(num_blocks):
            layers.append(BasicBlock(in_channels, out_channels, kernel_size=kernel_size, activation=act))
            in_channels = out_channels

        self.blocks = nn.Sequential(*layers)

        self.pool = nn.AdaptiveAvgPool1d(output_size=1)
        self.fc = nn.Linear(hidden_dim, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # x: [batch_size, n_features]
        batch_size, n_features = x.size()
        x = x.view(batch_size, 1, n_features)  # [batch_size, 1, n_features]

        out = self.conv_in(x)    # [batch_size, hidden_dim, n_features]
        out = self.blocks(out)   # [batch_size, hidden_dim, n_features]

        out = self.pool(out)     # [batch_size, hidden_dim, 1]
        out = out.squeeze(-1)    # [batch_size, hidden_dim]

        logit = self.fc(out)     # [batch_size, 1]
        prob = self.sigmoid(logit)
        return prob


class PyTorchResNetClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, num_blocks=2, hidden_dim=32, kernel_size=3, activation='relu',
                 learning_rate=0.001, max_iter=100, batch_size=64, device='cpu'):
        self.num_blocks = num_blocks
        self.hidden_dim = hidden_dim
        self.kernel_size = kernel_size
        self.activation = activation
        self.learning_rate = learning_rate
        self.max_iter = max_iter
        self.batch_size = batch_size
        self.device = device
        self.model_ = None
        self.n_features_ = None

    def fit(self, X, y):
        if hasattr(X, 'values'):
            X = X.values
        if hasattr(y, 'values'):
            y = y.values

        self.n_features_ = X.shape[1]
        X_tensor = torch.tensor(X, dtype=torch.float32).to(self.device)
        y_tensor = torch.tensor(y, dtype=torch.float32).view(-1,1).to(self.device)

        self.model_ = SimpleResNetClassifier(
            n_features=self.n_features_,
            num_blocks=self.num_blocks,
            hidden_dim=self.hidden_dim,
            kernel_size=self.kernel_size,
            activation=self.activation
        ).to(self.device)

        optimizer = optim.Adam(self.model_.parameters(), lr=self.learning_rate)
        criterion = nn.BCELoss()

        self.model_.train()
        for epoch in range(self.max_iter):
            for i in range(0, X_tensor.size(0), self.batch_size):
                batch_x = X_tensor[i:i+self.batch_size]
                batch_y = y_tensor[i:i+self.batch_size]

                optimizer.zero_grad()
                outputs = self.model_(batch_x)
                loss = criterion(outputs, batch_y)
                loss.backward()
                optimizer.step()

            if (epoch+1) % 10 == 0:
                print(f"Epoch [{epoch+1}/{self.max_iter}], Loss: {loss.item():.4f}")

        return self

    def predict_proba(self, X):
        if hasattr(X, 'values'):
            X = X.values
        self.model_.eval()
        with torch.no_grad():
            X_tensor = torch.tensor(X, dtype=torch.float32).to(self.device)
            pred = self.model_(X_tensor).cpu().numpy()
        return np.hstack([1 - pred, pred])

    def predict(self, X):
        proba = self.predict_proba(X)[:, 1]
        return (proba >= 0.5).astype(int)


resnet_clf = PyTorchResNetClassifier(
    num_blocks=2,
    hidden_dim=32,
    kernel_size=3,
    activation='relu',
    learning_rate=1e-3,
    max_iter=200,
    batch_size=64,
    device='cpu'
)


x_train_upsampled = x_train_upsampled.values if hasattr(x_train_upsampled, 'values') else x_train_upsampled
x_validation = x_validation.values if hasattr(x_validation, 'values') else x_validation
y_train_upsampled = y_train_upsampled.values if hasattr(y_train_upsampled, 'values') else y_train_upsampled
y_validation = y_validation.values if hasattr(y_validation, 'values') else y_validation

resnet_clf.fit(x_train_upsampled, y_train_upsampled)

y_pred = resnet_clf.predict(x_validation)
y_pred_proba = resnet_clf.predict_proba(x_validation)[:, 1]

acc = metrics.accuracy_score(y_validation, y_pred)
precision = metrics.precision_score(y_validation, y_pred, zero_division=0)
recall = metrics.recall_score(y_validation, y_pred, zero_division=0)
f1 = metrics.f1_score(y_validation, y_pred, zero_division=0)
auc = metrics.roc_auc_score(y_validation, y_pred_proba)
fpr, tpr, _ = metrics.roc_curve(y_validation, y_pred_proba)

results = [{
    'Label': "8:1:1",
    'Accuracy': "{:.3f}".format(acc),
    'Precision': "{:.3f}".format(precision),
    'Recall': "{:.3f}".format(recall),
    'ROC AUC': "{:.3f}".format(auc),
    'F1 Score': "{:.3f}".format(f1),
    'fpr': fpr,
    'tpr': tpr
}]

df_metrics = pd.DataFrame(results)
print(df_metrics[['Label', 'Accuracy', 'Precision', 'Recall', 'ROC AUC', 'F1 Score']])

fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(fpr, tpr, label=f"ROC AUC = {auc:.3f}", color='b')
ax.plot([0, 1], [0, 1], 'k--', lw=2, alpha=0.7)
ax.legend(loc="lower right")
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')
ax.set_title("ROC Curve for ResNet Model")
plt.show()